## Monte-Carlo type approximation to the true value of $π$

We have been tasked with finding the value of $π$ via a Monte-Carlo simulation, sounds simple enough but before we get to the programming, some basics:

### Monte-Carlo approximations briefly

We are talking about a technique used to estimate a numerical value by using random sampling, that is, simulating many random outcomes/samples and using their average to approximate the true value.
This is used instead of analytically solving the problem by exact calculations.

**Why it works?**

The Law of Large Numbers (LLN) -> As the number of samples ($N$) increases, $N \to \infty$, the value of our estimation converges to the true value, $\hat{\mu} \to \mu$.

The Central Limit Theorem (CLT) -> As the number of samples ($N$) increases, the error of our estimation decreases by $\frac{1}{\sqrt{N}}$, making it more accurate.

### Approximating the true value of $π$

Alright, now we know what are Monte-Carlo approximations, but how do we "aim for $π$" exactly?

We will do it with a **square** and a **circle**:

If we have a square of length $S$, it's area will be $S^2$

If we have a square of the same diameter, that is, of radius $R = \frac{S}{2}$, it's area will be $\pi R^2 = \pi\left(\tfrac{S}{2}\right)^2 = \tfrac{\pi S^2}{4}$

If we then overlap the square and circle, we can calculate the ratio of the square area that the circle occupies:
$$\frac{\tfrac{\pi S^2}{4}}{S^2} = \frac{\pi}{4}$$
And so, if we simulate a number of darts that are thrown at the square, calculate the proportion of them that fall inside the circle, and then multiply that number by 4, we will obtain $\pi$.

**Easy, let's do it!**

First, we import the necessary packages for our program

In [1]:
# We will use numpy for random number generation and array manipulations
import numpy as np

Now we start defining the functions that we will use for our simulation of $N$ samples

In [2]:

def shoot_darts(N: int, rng: np.random.Generator) -> tuple[np.ndarray, np.ndarray]:
    
    # Generates N random and independent x,y darts in the square [-1,1] x [-1,1]
    x = rng.uniform(-1.0, 1.0, size=N)
    y = rng.uniform(-1.0, 1.0, size=N)
    
    return x, y


def inside_circle(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    
    # Returns a boolean numpy array indicating whether each (x,y) dart falls inside the unit circle (radius=1, origin=(0,0))
    return (x*x + y*y) <= 1.0


def simulate_pi(N: int, seed: int | None = None) -> float:
    
    # Generates random numbers using the numpy random generator with a given seed if there is any (for reproducibility)
    rng = np.random.default_rng(seed)
    # Uses the function shoot_darts to get the x and y coordinates of the darts shot
    x, y = shoot_darts(N, rng)
    # Uses the function inside_circle to determine which darts are inside the unit circle
    inside = inside_circle(x, y)
    # Counts the number of darts that are inside the unit circle (that is, the number of True values in the boolean array)
    hits = np.count_nonzero(inside)
    # Calculates the proportion of darts that are inside the unit circle (EXPLAIN ---> mathematically this proportion is approximately π/4)
    proportion = hits / N
    # Returns the approximation of π
    return 4.0 * proportion

Now that we have the core functions, we define our final function that will run the experiment

In [3]:

def repeat_experiment(N: int, K: int, seed: int | None = None) -> dict:
    
    # This function repeats the simulation of π K times, each time with N darts, and returns a dictionary with the results
    
    # If there is no given seed, each run of the experiment will be independent and non-reproducible
    # If there is a given seed, we will use the numpy method to generate sub-seeds, this way each run of the experiment will be independent but reproducible
    if seed is None:
        seeds = [None] * K
    else:
        ss = np.random.SeedSequence(seed)
        seeds = ss.spawn(K)

    # Preallocated array to store the K estimates of π (avoids dynamic resizing)
    estimates = np.empty(K, dtype=float)

    for i in range(K):
        # If the seeds have been generated using SeedSequence, we create a new Generator for each run
        # Then we use the shoot_darts and inside_circle functions to estimate π as we explained before
        if isinstance(seeds[i], np.random.SeedSequence):
            rng_i = np.random.default_rng(seeds[i]) 
            x, y = shoot_darts(N, rng_i)
            hits = np.count_nonzero(inside_circle(x, y))
            estimates[i] = 4.0 * (hits / N)
        # If the seed is None, we just call simulate_pi with seed=None
        else:
            estimates[i] = simulate_pi(N, seed=seeds[i])

    # Here we calculate some descriptive statistics of the sample of k estimates of π
    mean = estimates.mean() # mean
    std = estimates.std(ddof=1) if K > 1 else 0.0 # standard deviation (applying Bessel's correction)
    se = std / np.sqrt(K) if K > 0 else np.nan # standard error of the mean
    ci95 = (mean - 1.96*se, mean + 1.96*se) if K > 0 else (np.nan, np.nan) # 95% confidence interval for the mean

    # Dictionary with the results tidily shown
    # To achieve that, we convert np.float64 to python native float (always accounting for NaN values), and restrict the number of decimals to 4
    result = {
        "first 10 estimates": [round(float(x), 4) for x in estimates[:10]],
        "mean": round(float(mean), 4),
        "std": round(float(std), 4) if not np.isnan(std) else float("nan"),
        "se": round(float(se), 4) if not np.isnan(se) else float("nan"),
        "ci95": (round(float(ci95[0]), 4), round(float(ci95[1]), 4))
                if not (np.isnan(ci95[0]) or np.isnan(ci95[1])) else (float("nan"), float("nan")),
    }
    return result

Alright, all done! Now let's run it and see what happens!

In [4]:
repeat_experiment(10, 10, seed=42)

{'first 10 estimates': [2.4, 2.8, 3.6, 2.8, 2.8, 2.4, 2.4, 3.6, 3.6, 3.6],
 'mean': 3.0,
 'std': 0.5416,
 'se': 0.1713,
 'ci95': (2.6643, 3.3357)}

We know that $\pi = 3.14159(...)$, so it seems that 10 samples of 10 darts each is not enough to obtain an accurate estimate, let's try with more!

In [5]:
repeat_experiment(100, 100, seed=42)

{'first 10 estimates': [3.24,
  2.84,
  3.08,
  3.12,
  2.96,
  2.96,
  3.12,
  3.48,
  3.0,
  3.24],
 'mean': 3.1548,
 'std': 0.1543,
 'se': 0.0154,
 'ci95': (3.1246, 3.185)}

Great, with 100 samples of 100 darts each we are getting real close, but let's add more!

In [6]:
repeat_experiment(1000, 1000, seed=42)

{'first 10 estimates': [3.192,
  3.1,
  3.12,
  3.184,
  3.232,
  3.052,
  3.208,
  3.22,
  3.1,
  3.18],
 'mean': 3.1422,
 'std': 0.0521,
 'se': 0.0016,
 'ci95': (3.1389, 3.1454)}

With the thousands we have 3 digits of $\pi$ correctly estimated, and the standard error is getting really small, but let's do it one more time.

In [7]:
repeat_experiment(10000, 10000, seed=42)

{'first 10 estimates': [3.1368,
  3.178,
  3.168,
  3.1616,
  3.1724,
  3.1412,
  3.1436,
  3.1492,
  3.1396,
  3.13],
 'mean': 3.1414,
 'std': 0.0164,
 'se': 0.0002,
 'ci95': (3.1411, 3.1418)}

We can see that for the tens of thousands the improvement has slowed down considerably. This is explained by the CLT, the "speed" at which our estimate approaches the true value is the same as the rate at which the standard error decreases, which is $\frac{1}{\sqrt{N}}$ and thus improves asymptotically.

Additionally, since $\pi$ is an irrational number, we will never obtain its exact value from a finite simulation, as the output is always rational. By contrast, if the true value was rational, like 0.4 for example, it would be possible to hit it exactly in a finite sample, but that wouldn't mean much as it would happen out of pure randomness.

The end